In [1]:
import os
import asyncio
import requests
import xmltodict
from typing import Dict, Any, List, Callable

from search_by_odk_api import get_odk_token
from mailer import send_email

In [18]:
def _export_participants(base_url: str, token: str, project_id: int) -> List[Dict[str, Any]]:
    """Return all entities from the `participantes` dataset."""
    headers = {"Authorization": f'Bearer {token}'}
    url = f'{base_url}/v1/projects/{project_id}/datasets/participantes/entities?extended=true'
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()

def _export_form_submissions(base_url: str, token: str, project_id: int, form_id: str) -> List[Dict[str, Any]]:
    """Return all submissions for a given form as parsed dicts."""
    headers = {"Authorization": f'Bearer {token}'}
    list_url = f'{base_url}/v1/projects/{project_id}/forms/{form_id}/submissions'
    resp = requests.get(list_url, headers=headers)
    resp.raise_for_status()
    submissions = []
    for sub in resp.json():
        xml_url = f'{list_url}/{sub["instanceId"]}.xml'
        xml_resp = requests.get(xml_url, headers=headers)
        if xml_resp.status_code != 200:
            continue
        parsed = xmltodict.parse(xml_resp.text)
        submissions.append(parsed.get('data', {}))
    return submissions

In [20]:
base_url = os.getenv('ODK_BASE_URL', '').rstrip('/')
project_id = int(os.getenv('ODK_PROJECT_ID', '2'))

token = get_odk_token()
if not token:
    raise RuntimeError('No se pudo obtener token')

print("Hola Mundo")
print(f'Token: {token}')

participants = _export_participants(base_url, token, project_id)
print(f'Participants: {len(participants)}')
# 
# form_ids = [
#     os.getenv('ODK_FORM_PREREGISTRO'),
#     os.getenv('ODK_FORM_CONSENT'),
#     os.getenv('ODK_FORM_EN_P1')
#     # os.getenv('ODK_FORM_EN_P2'),
#     # os.getenv('ODK_FORM_EN_P3'),
# ]
# 
# forms_data = {}
# for fid in form_ids:
#     if fid:
#         forms_data[fid] = _export_form_submissions(base_url, token, project_id, fid)
# 
# print(f'✔️ Participantes: {len(participants)} registros')
# for fid in forms_data:
#     print(f'✔️ Form {fid}: {len(forms_data[fid])} submissions')

✅ Token generado.
Hola Mundo
Token: jG1fiNbNxLG$BsoVOwqVlfjIXpz2V4apTDbsnOJf5Kkcs$20jUMbfKPjXzI7mtes
Participants: 119


In [4]:
print(forms_data.keys())
# print(forms_data.values())
print(forms_data.get('Laura2-piloto-encuesta-preregistro')[0])
print(forms_data.get('Laura2-piloto-encuesta-ic')[0])

dict_keys(['Laura2-piloto-encuesta-preregistro', 'Laura2-piloto-encuesta-ic', 'Laura2-piloto-encuesta-p1'])
{'@xmlns:jr': 'http://openrosa.org/javarosa', '@xmlns:orx': 'http://openrosa.org/xforms', '@id': 'Laura2-piloto-encuesta-preregistro', '@version': '20250704120248', 'participantes': {'note_image': None, 'nota_subtitutlo': None, 'nota_intro': None, 'edad': '20', 'mobile_phone': '988787772', 'participante_id': 'e0ebcd5220a84592564f19c9086a86ea1acf0d4a0f7a28428c05f78afeaf70f0', 'participante_id_counts': '0', 'exists_id': 'no', 'exists_message': 'Gracias por registrate.', 'mobile_phone_confirm': '988787772', 'mobile_match': 'yes', 'region': 'callao', 'correo': 'Dcortezdiaz28@gmail.com', 'correo_confirm': 'Dcortezdiaz28@gmail.com', 'motivacion': 'Saber con más profundidad sobre el proyecto', 'nota_2': None, 'short_id': 'e0ebcd'}, 'meta': {'instanceID': 'uuid:71a4896c-c5d6-4395-b9cc-93a6aea74997', 'instanceName': 'e0ebcd', 'entity': {'@dataset': 'participantes', '@id': '7c4cf8d7-f5f8-4

In [23]:
from utils import construir_url_consent
def correo_consentimiento(email, id_participant, short_id):
    # email = parsed["data"]["participantes"].get("correo")
    # id_participant = parsed["data"]["participantes"].get("participante_id")
    # short_id = parsed["data"]["participantes"].get("short_id")
    subject = "Proyecto Laura - Consentimiento informado"
    url_id = construir_url_consent(id_participant)
    message = f"""
                <p>Hola, gracias por tu interés en participar en el proyecto Laura.</p>

                <p>Ahora  que has completado el formulario de pre-registro, hemos generado un código de participante para tí</p>

                <p>{short_id}</p>

                <p>Este código permitirá proteger tu identidad, ya que podrás utilizarlo para identificarte en futuras interacciones dentro del proyecto sin dar tu nombre o apellido.</p>

                <p>Ahora continuemos con algo muy importante, el <strong>Consentimiento Informado</strong> <a href="https://drive.google.com/file/d/1rgvpfLpdQvESCBBQGlnZRyxP4wscF3X2/view?usp=sharing"><strong>(leer aquí)</strong></a>, para tu mayor comodidad también hemos realizado un video que lo explica <a href="https://drive.google.com/file/d/1Z_jL6Zjr-295Sd5mI5xPt9Nd5UP_-COI/view?usp=drive_link">(ver video aquí)</a>.</p>

                <p>Ya estás decidida a participar?, entonces ahora completa el formulario de consentimiento informado <a href={url_id}>(completar aquí)</a>, luego recibirás un correo con los enlaces de la encuesta</p>

                <p>Tu participación ayudará a que instituciones y tomadores de decisiones de todo el país conozcan los principales problemas de salud que aquejan a la mujer peruana.</p>


                <p>Atentamente,<br>
                Equipo del proyecto Laura</p>
                <p><img src="https://drive.google.com/uc?export=view&id=109KJ3wBlPtuv5uc1QsM3igm61v6OO00O" alt="Logo LAURA" width="150"/></p>
            """

    return email, subject, message

In [26]:
from search_by_odk_api import buscar_datos_en_entidad_participantes
from mailer import send_email
for entity in participants:
    if entity.get('currentVersion').get('label',{}) == '928811762':
        data = buscar_datos_en_entidad_participantes(entity.get('currentVersion').get('label',{}))
        print(data)
        email = data['email']
        id_participant = data['long_id']
        short_id = data['short_id']
        email, subject, message = correo_consentimiento(email, id_participant, short_id)
        print(f"✅ Enviando correo a {email}")
        await send_email(subject=subject, html_message=message, recipient=email)
        print(f"✅ Correo enviado a {email}")
        # print(entity, entity.get('currentVersion').get('label',{}))
    # print(entity.get('currentVersion').get('label',{}))
    # if entity.get('currentVersion').get('label') == '93047375':
    # print(entity, entity.get('currentVersion').get('label'))
    # data = entity.get('currentVersion', {})
    # print(entity)

✅ Token generado.
✅ Posible entidad encontrada con label: 928811762 (UUID: 3b0757bf-c243-41ca-94f5-136c626cdf41)
{'email': 'luceroillatopa7@gmail.com', 'phone': '928811762', 'consent': 'yes', 'long_id': '72fd0cb884a0870b21e7bc66de1bd57ab97aadf80d7979b096866b27891d4997', 'short_id': '72fd0c'}
{'email': 'luceroillatopa7@gmail.com', 'phone': '928811762', 'consent': 'yes', 'long_id': '72fd0cb884a0870b21e7bc66de1bd57ab97aadf80d7979b096866b27891d4997', 'short_id': '72fd0c'}
✅ Enviando correo a luceroillatopa7@gmail.com
✅ Correo enviado a luceroillatopa7@gmail.com


In [12]:
def filtrar_sin_consentimiento(data: Dict[str, Any], forms: Dict[str, List[Dict[str, Any]]]) -> bool:
    phone = data.get('currentVersion').get('label')
    print(f'Telefono: {phone}')
    if not phone:
        return False
    consent_form_id = os.getenv('ODK_FORM_CONSENT', 'Laura2-piloto-encuesta-ic')
    consent_subs = forms.get(consent_form_id, [])
    for sub in consent_subs:
        if sub.get('preamble', {}).get('entity_phone') == phone:
            return False
    return True

In [15]:
filtered = []
for entity in participants:
    if filtrar_sin_consentimiento(entity, forms_data):
        filtered.append(entity)
# print(filtered)
print(len(filtered))

Telefono: 925248146
Telefono: 953459304
Telefono: 996473920
Telefono: 935231110
Telefono: 936510292
Telefono: 968840501
Telefono: 926960649
Telefono: 902200303
Telefono: 969659778
Telefono: 940397318
Telefono: 981589813
Telefono: 970336485
Telefono: 912568430
Telefono: 930473750
Telefono: 934715515
Telefono: 948061243
Telefono: 995179389
Telefono: 979192997
Telefono: 958998097
Telefono: 902121965
Telefono: 942146269
Telefono: 992190761
Telefono: 954791776
Telefono: 956620907
Telefono: 992089137
Telefono: 922028518
Telefono: 928811762
Telefono: 987245131
Telefono: 916318475
Telefono: 935061039
Telefono: 910483910
Telefono: 920525235
Telefono: 989914160
Telefono: 901780361
Telefono: 991362755
Telefono: 991838251
Telefono: 942434945
Telefono: 982567520
Telefono: 992025191
Telefono: 927518740
Telefono: 983438137
Telefono: 973422366
Telefono: 965641547
Telefono: 902111049
Telefono: 946013286
Telefono: 903132690
Telefono: 962927230
Telefono: 931859882
Telefono: 910384969
Telefono: 998879958


In [16]:
for i in filtered:
    print(i.get('currentVersion').get('label'), i.get('createdAt'))

925248146 2025-07-04T03:22:43.418Z
953459304 2025-07-02T23:40:57.444Z
969659778 2025-06-26T17:59:21.964Z
981589813 2025-06-26T01:29:46.822Z
930473750 2025-06-24T22:17:49.024Z
942146269 2025-06-22T18:13:59.973Z
992190761 2025-06-22T17:08:19.491Z
992089137 2025-06-21T23:46:51.167Z
916318475 2025-06-21T01:02:38.496Z
935061039 2025-06-21T00:23:58.723Z
910483910 2025-06-20T23:49:11.568Z
920525235 2025-06-20T23:49:11.524Z
989914160 2025-06-20T23:49:11.502Z
901780361 2025-06-20T23:49:11.414Z
991838251 2025-06-20T23:49:11.355Z
942434945 2025-06-19T03:13:20.296Z
982567520 2025-06-18T02:39:38.674Z
992025191 2025-06-17T04:29:51.817Z
927518740 2025-06-16T19:29:04.197Z
983438137 2025-06-16T17:54:24.676Z
902111049 2025-06-15T03:19:26.148Z
931859882 2025-06-13T23:51:25.224Z
958175388 2025-06-12T19:32:18.659Z
976338158 2025-06-12T02:19:37.288Z
935295036 2025-06-12T01:31:23.029Z
904505623 2025-06-11T11:49:12.253Z
921417184 2025-06-10T18:50:12.859Z
923183882 2025-06-09T18:21:09.919Z
953700289 2025-06-09

In [4]:
print(forms_data.get('Laura2-piloto-encuesta-ic')[0])
print(forms_data.get('Laura2-piloto-encuesta-p1')[0])

{'@xmlns:jr': 'http://openrosa.org/javarosa', '@xmlns:orx': 'http://openrosa.org/xforms', '@id': 'Laura2-piloto-encuesta-ic', '@version': '20250628174727', 'preamble': {'entity_name': None, 'entity_phone': None, 'entity_email': None, 'entity_short_id': None, 'note_image': None, 'part_id': 'b72a09d682b460ea87720b1f2bd59afb9956707c07e041c57d9f7f59b861763c', 'note_consent': None}, 'consent': {'note_welcome': None, 'note_image': None, 'note_consent_0': None, 'Q0_read_consent': 'yes', 'Q0_accept_consent': 'yes', 'Q1_accept_consent': 'yes', 'Q2_accept_recontact': 'yes'}, 'meta': {'instanceID': 'uuid:7d7b9d3e-479b-4a6c-b3bc-ca3cb45c31e7'}}
{'@xmlns:jr': 'http://openrosa.org/javarosa', '@xmlns:orx': 'http://openrosa.org/xforms', '@id': 'Laura2-piloto-encuesta-p1', '@version': '20250628233628', 'preamble': {'entity_name': 'ed737023-fbc3-444b-85de-06d1433376f0', 'entity_phone': '926960649', 'entity_email': 'giulianaponce26@gmail.com', 'entity_short_id': '43ee63', 'note_image': None, 'part_id_2':

In [5]:
def filtrar_sin_en(data: Dict[str, Any], forms: Dict[str, List[Dict[str, Any]]]) -> bool:
    entity_id = data.get('preamble').get('entity_name')
    
    # print(f'ID Entidad: {entity_id}')
    if not entity_id:
        return False
    en_form_id = 'Laura2-piloto-encuesta-p1'
    en_subs = forms.get(en_form_id, [])
    for sub in en_subs:
        if sub.get('preamble', {}).get('entity_name') == entity_id:
            # print(f'ID Entidad: {entity_id}')
            return False
    return True

In [6]:
filtered = []
consent_subs = forms_data.get('Laura2-piloto-encuesta-ic')
for subs in consent_subs:
    if filtrar_sin_en(subs, forms_data):
        filtered.append(subs)
# print(filtered)
print(len(filtered))

29


In [7]:
print(filtered[0])

{'@xmlns:jr': 'http://openrosa.org/javarosa', '@xmlns:orx': 'http://openrosa.org/xforms', '@id': 'Laura2-piloto-encuesta-ic', '@version': '20250628174727', 'preamble': {'entity_name': 'a56b141b-84fa-4cfa-ad88-1a89d3cebb59', 'entity_phone': '980338929', 'entity_email': 'jhakeline.30.01.02@gmail.com', 'entity_short_id': '6722bf', 'note_image': None, 'part_id': '6722bf66bf70390e4907e4c2776f489a9e8c9b89ffeaa8e9dc351515c89aa379', 'note_consent': None}, 'consent': {'note_welcome': None, 'note_image': None, 'note_consent_0': None, 'Q0_read_consent': 'yes', 'Q0_accept_consent': 'yes', 'Q1_accept_consent': 'yes', 'Q2_accept_recontact': 'yes'}, 'meta': {'instanceID': 'uuid:7f0f38d3-5c98-4d04-9f2d-3c151837e236'}}


In [10]:
def correo_encuesta_nac(participant_id, parsed):
    # email = buscar_correo_en_submissions(participant_id)
    # edad = buscar_edad_en_submissions(participant_id)
    email = parsed["preamble"].get("entity_email")
    short_id = parsed["preamble"].get("entity_id")
    # phone = parsed["data"]["preamble"].get("entity_phone")
    # datos = buscar_datos_en_entidad_participantes(phone)
    # short_id = datos.get("short_id")
    url_p1 = construir_url_part1(participant_id)
    url_p2 = construir_url_part2(participant_id)
    url_p3 = construir_url_part3(participant_id)
    # print("🔎 participant_id:", participant_id)
    subject = f"¡Gracias por completar el Consentimiento Informado del proyecto Laura!"
    message = f"""
                    <p>Hola {short_id},</p>

                    <p>Hemos recibido tu consentimiento informado para participar en este estudio. 🎉</p>

                    <p>Ya podemos empezar con la <strong>Encuesta Nacional</strong>, la que hemos dividido en 3 bloques.</p>

                    <p><strong>¡Importante!</strong>👀 Una vez que hayas iniciado cada bloque, el sistema solo guarda las respuestas cuando lo hayas terminado y <strong>enviado</strong>, por eso te pedimos que destines un momento del día para completarlo. Si sales antes de completarlo podrías perder lo que has avanzado.</p>


                    <li>
                        Formulario 1 - Datos Generales: <a href={url_p1}>Acceder</a> - Llena tus respuestas y presiona <strong>enviar</strong>. Ahora continúa con el bloque 2.
                    </li>
                    <li>
                        Formulario 2 - Salud Reproductiva y Menstrual: <a href={url_p2}>Acceder</a> - Llena tus respuestas y presiona <strong>enviar</strong>. Ahora continúa con el bloque 3.
                    </li>
                    <li>
                        Formulario 3 - Salud Mental: <a href={url_p3}>Acceder</a> - Llena tus respuestas y presiona <strong>enviar</strong>, ya terminaste!
                    </li>

                    <p>Muchas gracias por tu participación en el proyecto <strong>Laura</strong>. 🫶</p>

                    <p>Atentamente,<br>
                    Equipo del proyecto Laura</p>

                    <p><img src="https://drive.google.com/uc?export=view&id=109KJ3wBlPtuv5uc1QsM3igm61v6OO00O" alt="Logo LAURA" width="150"/></p>
                """
    return email, subject, message

def construir_url_part1(valor_id):

    # Prueba
    # https://odkcentral.upch.edu.pe/-/single/oJaqbizarAl2a5ITzH2YsX2gjzETtZc?st=Ai6eTLM1bVT0MnYHivkTxXejfKiJLISTiexXD6hZF9rLr39ilDt6PS$n0zV4VbAG

    # https://odkcentral.upch.edu.pe/-/single/guiLDqa7lfyyWCBhv9k2AWvqqMuPni6?st=Zm78egptTVNqykyl2UY57k8RCh5n9l6YBilieqsg2Z0jtGuXrg2OcY$IPTwARukQ
    formulario = {
        "form_id": "guiLDqa7lfyyWCBhv9k2AWvqqMuPni6",
        "token": "Zm78egptTVNqykyl2UY57k8RCh5n9l6YBilieqsg2Z0jtGuXrg2OcY$IPTwARukQ",
        "part_id": "d[/data/preamble/part_id_2]"
        # "age": "d[/data/general_data/Q1.3_age]"
        }
    return f"https://odkcentral.upch.edu.pe/-/single/{formulario['form_id']}?st={formulario['token']}&{formulario['part_id']}={valor_id}"

def construir_url_part2(valor_id):

    # Prueba
    # https://odkcentral.upch.edu.pe/-/single/fgLy1rY5M1YOeycvuIxqMHsUNBYoZCX?st=l65vr7s7G80yoQAWHGnWXHOIWT!JeudSgB6CfxhOMIow4LK7rirRypW!mExW!0g2

    # https://odkcentral.upch.edu.pe/-/single/xZI6VypZiTI71JQ1YlzKBDsqG6ahzKv?st=UZk8rikW3mRRn3Ic7UidcfvWb3$uNeK0oZkIvi0JGUGSRufhyv1FzVzCt7BbFUkW
    formulario = {
        "form_id": "xZI6VypZiTI71JQ1YlzKBDsqG6ahzKv",
        "token": "UZk8rikW3mRRn3Ic7UidcfvWb3$uNeK0oZkIvi0JGUGSRufhyv1FzVzCt7BbFUkW",
        "part_id": "d[/data/preamble/part_id_3]"
        }
    return f"https://odkcentral.upch.edu.pe/-/single/{formulario['form_id']}?st={formulario['token']}&{formulario['part_id']}={valor_id}"

def construir_url_part3(valor_id):

    # Prueba
    # https://odkcentral.upch.edu.pe/-/single/vmngcom1ZaTITHFj5MHfJefN6Oevjk0?st=pNFemRmzNQmwZ8hYzdSo3ttz$jFNlIm9PKCcygRm9duzJOC6CJ3vIGvt1NI5dYm4

    # https://odkcentral.upch.edu.pe/-/single/CvCLsSXXVpoL537Bati0fXXrFIcj8MJ?st=Gp6hXKGnARJC7f4X4ybzXHQYt2VGIh!FPiss8N$MKLrXzZ69oHWmv8segsPNCp5u
    formulario = {
        "form_id": "CvCLsSXXVpoL537Bati0fXXrFIcj8MJ",
        "token": "Gp6hXKGnARJC7f4X4ybzXHQYt2VGIh!FPiss8N$MKLrXzZ69oHWmv8segsPNCp5u",
        "part_id": "d[/data/preamble/part_id_4]"
        }
    return f"https://odkcentral.upch.edu.pe/-/single/{formulario['form_id']}?st={formulario['token']}&{formulario['part_id']}={valor_id}"


In [12]:
from mailer import send_email
for i in filtered:
    participant_id = i["preamble"].get("part_id")  # <-- Aquí está el valor que vino vía prellenado
    consentimiento = i["consent"].get("Q0_accept_consent")
    print("🔎 participant_id (desde part_id):", participant_id)
    if consentimiento == "yes":
        email, subject, message = correo_encuesta_nac(participant_id, i)
        print(email)
            # if email:
            #     print(f"✅ Enviando correo a {email}")
            #     await send_email(subject=subject, html_message=message, recipient=email)
            #     print(f"✅ Correo enviado a {email}")
            # else:
            #     print(f"⚠️ No se encontró correo para ID: {participant_id}")

🔎 participant_id (desde part_id): 6722bf66bf70390e4907e4c2776f489a9e8c9b89ffeaa8e9dc351515c89aa379
jhakeline.30.01.02@gmail.com
🔎 participant_id (desde part_id): 9027f121d3c7d883caec8630f73b11d0e4fe19662874dc5078d79c105abda079
None
🔎 participant_id (desde part_id): c0cd3761fa1a5aaed7610d3b2220af819362b41d237d0d4f4a471f620263d6d3
None
🔎 participant_id (desde part_id): c0cd3761fa1a5aaed7610d3b2220af819362b41d237d0d4f4a471f620263d6d3
None
🔎 participant_id (desde part_id): 3d82b449aa511295f7c1e48e25d293628cef86607c2e6d224537f60b9001b1a1
None
🔎 participant_id (desde part_id): 4549fee0d32ffedd63c6184b2624e916344f2c8010e997584a382edccd25fd14
None
🔎 participant_id (desde part_id): 608f3883a91bce5d749869ba706f635e84e7825e7fab0a732e6fc512b433326e
None
🔎 participant_id (desde part_id): 6722bf66bf70390e4907e4c2776f489a9e8c9b89ffeaa8e9dc351515c89aa379
None
🔎 participant_id (desde part_id): e45ddd3ed04532a93c5c0d3fd6f42a1fa3b44d923219f88deaa7bc6df7ca7a6b
None
🔎 participant_id (desde part_id): 72fd0c

## 3. Enviar correos a los participantes filtrados

In [ ]:
# DEFAULT_SUBJECT = 'Proyecto Laura - Notificacion'
# DEFAULT_MESSAGE = (
#     '<p>Hola,</p><p>Este es un mensaje generado automaticamente desde el entorno local del proyecto Laura.</p>'
# )
# 
# for entity in participants:
#     data = entity.get('currentVersion', {}).get('data', {})
#     if filtrar_por_ejemplo(data):
#         email = data.get('correo') or data.get('email')
#         if email:
#             await send_email(DEFAULT_SUBJECT, DEFAULT_MESSAGE, email)